## STEP 1. ANOVA 분석

In [102]:
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multicomp import MultiComparison

# 파일 불러오기
file_path = './ANOVA_rename.csv'
df = pd.read_csv(file_path)

#df = df[~df['Class'].isin(['QC', 'Control'])]  ## QC/Control 제외, 즉 정확하게 해당 글자로 타이핑 해야함. 대소문자 구별 주의

In [103]:
df.shape

(190, 7)

In [104]:
# 결측값 제거
df = df.dropna()
df.shape

(169, 7)

In [105]:
# 결과를 저장할 리스트 초기화
results = []

In [106]:
# 'Class'가 그룹이고, 각 protein column이 분석 대상
group_column = 'Class'
protein_columns = df.columns[2:]  # protein columns

# 각 protein에 대해 ANOVA 수행
for protein in protein_columns:
    # 그룹별로 데이터 추출 (결측값은 이미 제거됨)
    groups = [df[protein][df[group_column] == group] for group in df[group_column].unique()]
    
    # 그룹별 데이터가 존재하는 경우에만 ANOVA 수행
    if all([len(group) > 0 for group in groups]):
        f_stat, p_value = stats.f_oneway(*groups)
        results.append({'Protein': protein, 'F-value': f_stat, 'P-value': p_value})
    else:
        results.append({'Protein': protein, 'F-value': None, 'P-value': None})

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

In [107]:
results_df

,Protein,F-value,P-value
0,Protein_1,3.923470,0.004542
1,Protein_2,1.689493,0.154831
2,Protein_3,1.698392,0.152796
3,Protein_4,1.902700,0.112393
4,Protein_5,4.583375,0.001556


In [108]:

results_df.to_csv('000.anova_results.csv', index=False)  ## 결과 저장

## STEP 2. p-value 선별

In [109]:
# P-value가 0.05 이하인 protein만 필터링
significant_proteins = results_df[results_df['P-value'] < 0.05]['Protein']

In [110]:
significant_proteins

0    Protein_1
4    Protein_5
Name: Protein, dtype: object

## STEP 3. 사후분석

In [111]:
tukey_results = []
bonf_results = []

for protein in significant_proteins:
    
    # Tukey HSD
    mc = MultiComparison(df[protein], df[group_column])
    tukey_result = mc.tukeyhsd(alpha=0.05)

    for comparison in tukey_result.summary().data[1:]:
        group1, group2, meandiff, p_adj, lower, upper, reject = comparison
        tukey_results.append({
            'Protein': protein,
            'Group1': group1,
            'Group2': group2,
            'p-value adj': p_adj,
            'Method': 'Tukey'
        })

    # Bonferroni
    bonf_result = mc.allpairtest(stats.ttest_ind, method="bonf")[0]

    for comparison in bonf_result.data[1:]:
        group1, group2, stat, pval, pval_corr, reject = comparison
        bonf_results.append({
            'Protein': protein,
            'Group1': group1,
            'Group2': group2,
            'p-value corr': pval_corr,
            'Method': 'Bonferroni'
        })


#### 사후분석 Resutl ####
tukey_df = pd.DataFrame(tukey_results)
bonf_df = pd.DataFrame(bonf_results)


In [112]:

# 결과를 각각 Excel 파일로 저장
tukey_df.to_csv('./001.tukey_analysis_results.csv', index=False)
bonf_df.to_csv('./001.bonferroni_analysis_results.csv', index=False)

## STEP 4. 사후분석 선별

In [116]:
import pandas as pd

# 파일 불러오기
bonferroni_file_path = './001.bonferroni_analysis_results.csv'
tukey_file_path = './001.tukey_analysis_results.csv'

# Excel 파일 읽기
bonf_df = pd.read_csv(bonferroni_file_path)
tukey_df = pd.read_csv(tukey_file_path)

# 0.05 이하의 p-value adjustment 값을 가진 행만 필터링
filtered_bonf_df = bonf_df[bonf_df['p-value corr'] < 0.05]
filtered_tukey_df = tukey_df[tukey_df['p-value adj'] < 0.05]

# 필터링된 결과를 각각 새로운 Excel 파일로 저장
filtered_bonf_df.to_csv('./002.filtered_bonferroni_results.csv', index=False)
filtered_tukey_df.to_csv('./002.filtered_tukey_results.csv', index=False)
